In [ ]:
from zipfile import ZipFile

In [ ]:
check = ZipFile("chunk1.zip").namelist()
print(check[:5])

#### Classe PeerExtractor:

Gestisce l'estrazione e la decompressione dei file JSON dall'archivio ZIP.
Contiene il metodo statico process_json_data per processare i dati JSON decompressi.

#### Classe OciProcess:

Inizializza e gestisce il dizionario di lookup per i codici OCI.
Include metodi per calcolare il prossimo codice di lookup, aggiornare il dizionario di lookup, e convertire un DOI in un codice CI.
#### Classe CSVWriter:

Gestisce la scrittura dei dati degli articoli su file CSV.
Utilizza la classe OciProcess per generare i codici OCI per ogni articolo.

#### Metodo parse_date:

Funzione di supporto per il parsing delle date.
Questo approccio orientato agli oggetti migliora la modularità e la manutenibilità del codice, rendendo più facile l'estensione e la gestione delle singole componenti del processo.

In [ ]:
import zipfile
import gzip
import json
import csv
import os
import errno
import argparse
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm
import polars as pl

LOOKUP_CSV = 'lookup.csv'
CROSSREF_CODE = '020'

class PeerExtractor:
    def __init__(self, zip_filename, batch_size=10):
        self.zip_filename = zip_filename
        self.batch_size = batch_size

    def process_files(self, csv_writer, max_files=None):
        with zipfile.ZipFile(self.zip_filename, 'r') as zip_file:
            file_infos = [file_info for file_info in zip_file.infolist() if file_info.filename.endswith(".json.gz")]
            
            if max_files:
                file_infos = file_infos[:max_files]
            
            for batch in self.batch(file_infos, self.batch_size):
                peer_review_items = self.process_batch(zip_file, batch)
                csv_writer.write_to_csv(peer_review_items)

    def batch(self, iterable, n=1):
        length = len(iterable)
        for ndx in range(0, length, n):
            yield iterable[ndx:min(ndx + n, length)]

    def process_batch(self, zip_file, batch):
        peer_review_items = []
        with ThreadPoolExecutor(max_workers=2) as executor:
            futures = [executor.submit(self.process_file, zip_file, file_info) for file_info in batch]
            for future in tqdm(as_completed(futures), total=len(futures), desc="Processing batch"):
                peer_review_items.extend(future.result())
        return peer_review_items

    def process_file(self, zip_file, file_info):
        with zip_file.open(file_info) as compressed_file:
            compressed_data = compressed_file.read()
            return self.process_json_data(compressed_data)

    def process_json_data(self, compressed_data):
        decompressed_data = gzip.decompress(compressed_data)
        decoded_data = decompressed_data.decode('utf-8')
        
        try:
            json_data = json.loads(decoded_data)
        except json.JSONDecodeError:
            print("Errore di decodifica JSON per il seguente dato: ", decoded_data)
            return []

        if isinstance(json_data, dict) and 'items' in json_data:
            items = json_data['items']
        elif isinstance(json_data, list):
            items = json_data
        else:
            print("Struttura JSON non riconosciuta: ", json_data)
            return []

        peer_review_items = [item for item in items if item.get('type') == 'peer-review']
        return peer_review_items

class OciProcess:
    def __init__(self, lookup_csv=LOOKUP_CSV, crossref_code=CROSSREF_CODE):
        self.lookup_code = 0
        self.lookup_dic = {}
        self.LOOKUP_CSV = lookup_csv
        self.CROSSREF_CODE = crossref_code
        self.init_lookup_dic()

    def init_lookup_dic(self):
        with open(self.LOOKUP_CSV, 'r', encoding='utf-8') as lookupcsv:
            lookupcsv_reader = csv.DictReader(lookupcsv)
            code = -1
            for row in lookupcsv_reader:
                if row['c'] not in self.lookup_dic:
                    self.lookup_dic[row['c']] = row['code']
                    code = int(row['code'])
            self.lookup_code = code

    def calc_next_lookup_code(self):
        rem = self.lookup_code % 100
        newcode = self.lookup_code + 1
        if rem == 89:
            newcode = (self.lookup_code // 100 + 1) * 100
        self.lookup_code = newcode

    def update_lookup(self, c):
        if c not in self.lookup_dic:
            self.calc_next_lookup_code()
            code = str(self.lookup_code).zfill(2)
            self.lookup_dic[c] = code
            self.write_txtblock_on_csv(self.LOOKUP_CSV, '\n"%s","%s"' % (c, code))

    def write_txtblock_on_csv(self, csv_path, block_txt):
        directory = os.path.dirname(csv_path)
        if directory and not os.path.exists(directory):
            try:
                os.makedirs(directory)
            except OSError as exc:
                if exc.errno != errno.EEXIST:
                    raise
        with open(csv_path, 'a', newline='', encoding='utf-8') as csvfile:
            csvfile.write(block_txt)

    def convert_doi_to_ci(self, doi_str):
        return self.CROSSREF_CODE + self.match_str_to_lookup(doi_str)

    def match_str_to_lookup(self, str_val):
        ci_str = ""
        str_noprefix = str_val[3:]
        for c in str_noprefix:
            if c not in self.lookup_dic:
                self.update_lookup(c)
            ci_str += str(self.lookup_dic[c])
        return ci_str

class CSVWriter:
    def __init__(self, output_filenames):
        if isinstance(output_filenames, str):
            self.output_filenames = [output_filenames]
        else:
            self.output_filenames = output_filenames

    def write_to_csv(self, peer_review_items):
        for output_filename in self.output_filenames:
            with open(output_filename, 'a', newline='', encoding='utf-8') as output_file:
                fieldnames = ["oci", "citing_doi", "cited_doi", "citing_date", "citing_url"]
                writer = csv.DictWriter(output_file, fieldnames=fieldnames)
                if output_file.tell() == 0:
                    writer.writeheader()

                oci_processor = OciProcess()

                for element in peer_review_items:
                    for i in element.get("relation", {}).get("is-review-of", []):
                        doi_p = element["DOI"]
                        doi_a = i.get("id", "")
                        url_p = element["URL"]
                        citing_entity_local_id = oci_processor.convert_doi_to_ci(doi_p)
                        cited_entity_local_id = oci_processor.convert_doi_to_ci(doi_a)
                        oci = "oci:" + citing_entity_local_id + "-" + cited_entity_local_id
                        date_peer_review = str(element["created"]["date-time"])[:10]
                        if doi_p and doi_a:
                            writer.writerow({
                                "oci": oci,
                                "citing_doi": doi_p,
                                "cited_doi": doi_a,
                                "citing_date": date_peer_review,
                                "citing_url": url_p
                            })
            print("peer items saved to", output_filename)

    def remove_duplicates(self, input_filename, output_filename):
        df = pl.read_csv(input_filename)
        df_unique = df.unique(subset=['oci'])
        df_unique.write_csv(output_filename)
        print(f"Unique peer items saved to {output_filename}")

def main():
    parser = argparse.ArgumentParser(description="Process JSON.gz files in a ZIP and output to CSV.")
    parser.add_argument("zip_filename", help="The input ZIP file containing JSON.gz files.")
    parser.add_argument("output_filenames", help="The output CSV file(s).", nargs='+')
    parser.add_argument("--batch_size", type=int, default=10, help="Number of files to process in each batch.")
    parser.add_argument("--max_files", type=int, help="Maximum number of files to process.")

    args = parser.parse_args()

    csv_writer = CSVWriter(args.output_filenames)
    article_processor = PeerExtractor(args.zip_filename, args.batch_size)
    article_processor.process_files(csv_writer, args.max_files)

    for output_filename in args.output_filenames:
        unique_output_filename = output_filename.replace(".csv", "_unique.csv")
        csv_writer.remove_duplicates(output_filename, unique_output_filename)

if __name__ == "__main__":
    main()


In [ ]:
import zipfile
import gzip
import json
import csv
from concurrent.futures import ThreadPoolExecutor, as_completed
import argparse
from tqdm import tqdm

class NonPeerExtractor:
    def __init__(self, zip_filename, batch_size=10):
        self.zip_filename = zip_filename
        self.batch_size = batch_size

    def process_files(self, csv_writer, max_files=None):
        with zipfile.ZipFile(self.zip_filename, 'r') as zip_file:
            file_infos = [file_info for file_info in zip_file.infolist() if file_info.filename.endswith(".json.gz")]
            
            if max_files:
                file_infos = file_infos[:max_files]
            
            for batch in self.batch(file_infos, self.batch_size):
                non_peer_review_items = self.process_batch(zip_file, batch)
                csv_writer.write_to_csv(non_peer_review_items)

    def batch(self, iterable, n=1):
        length = len(iterable)
        for ndx in range(0, length, n):
            yield iterable[ndx:min(ndx + n, length)]

    def process_batch(self, zip_file, batch):
        non_peer_review_items = []
        with ThreadPoolExecutor(max_workers=2) as executor:
            futures = [executor.submit(self.process_file, zip_file, file_info) for file_info in batch]
            for future in tqdm(as_completed(futures), total=len(futures), desc="Processing batch"):
                non_peer_review_items.extend(future.result())
        return non_peer_review_items

    def process_file(self, zip_file, file_info):
        with zip_file.open(file_info) as compressed_file:
            compressed_data = compressed_file.read()
            return self.process_json_data(compressed_data)

    def process_json_data(self, compressed_data):
        decompressed_data = gzip.decompress(compressed_data)
        decoded_data = decompressed_data.decode('utf-8')
        
        try:
            json_data = json.loads(decoded_data)
        except json.JSONDecodeError:
            print("Errore di decodifica JSON per il seguente dato: ", decoded_data)
            return []

        if isinstance(json_data, dict) and 'items' in json_data:
            items = json_data['items']
        elif isinstance(json_data, list):
            items = json_data
        else:
            print("Struttura JSON non riconosciuta: ", json_data)
            return []

        non_peer_review_items = [item for item in items if item.get('type') != 'peer-review']
        return non_peer_review_items

class CSVWriter:
    def __init__(self, output_filenames):
        if isinstance(output_filenames, str):
            self.output_filenames = [output_filenames]
        else:
            self.output_filenames = output_filenames
        self.header_written = False

    def write_to_csv(self, non_peer_review_items):
        for output_filename in self.output_filenames:
            with open(output_filename, 'a', newline='', encoding='utf-8') as output_file:
                fieldnames = ['cited_doi', 'cited_url', 'cited_issn', 'cited_venue', 'cited_date']
                writer = csv.DictWriter(output_file, fieldnames=fieldnames)
                if output_file.tell() == 0:
                    writer.writeheader()

                for element in non_peer_review_items:
                    doi_a = element.get("DOI")
                    url_a = element.get("URL")
                    date_non_peer_review = str(element.get("created", {}).get("date-time", ""))[:10]
                    issn = ', '.join(element.get('ISSN', []))
                    container_title = ', '.join(element.get('container-title', []))
                    if doi_a and url_a:
                        writer.writerow({
                            "cited_doi": doi_a,
                            "cited_url": url_a,
                            "cited_issn": issn,
                            "cited_venue": container_title,
                            "cited_date": date_non_peer_review
                        })
            print("Batch saved to", output_filename)

def main():
    parser = argparse.ArgumentParser(description="Process JSON.gz files in a ZIP and output to CSV.")
    parser.add_argument("zip_filename", help="The input ZIP file containing JSON.gz files.")
    parser.add_argument("output_filenames", help="The output CSV file(s).", nargs='+')
    parser.add_argument("--batch_size", type=int, default=10, help="Number of files to process in each batch.")
    parser.add_argument("--max_files", type=int, help="Maximum number of files to process.")

    args = parser.parse_args()

    csv_writer = CSVWriter(args.output_filenames)
    article_processor = NonPeerExtractor(args.zip_filename, args.batch_size)
    article_processor.process_files(csv_writer, args.max_files)

if __name__ == "__main__":
    main()


### Filtering dataframes

In [ ]:
import argparse
import polars as pl
from datetime import datetime
import pytz
from dateutil.parser import parse
from dateutil.relativedelta import relativedelta

class Filter:
    def __init__(self, peer_review_paths, non_peer_review_paths, output_path, column_to_join="cited_doi"):
        self.peer_review_paths = peer_review_paths
        self.non_peer_review_paths = non_peer_review_paths
        self.output_path = output_path
        self.column_to_join = column_to_join

    def read_and_concatenate_dataframes(self, paths):
        dataframes = [pl.read_csv(path) for path in paths]
        concatenated_df = pl.concat(dataframes)
        return concatenated_df

    def validate_dataframes(self, df1, df2):
        if self.column_to_join not in df1.columns or self.column_to_join not in df2.columns:
            raise ValueError(f"Column {self.column_to_join} is not present in both DataFrames.")

    def join_dataframes(self, df1, df2):
        joined_df = df1.join(df2, on=self.column_to_join, how="inner")
        return joined_df

    def add_provenance(self, df):
        # Define provenance information
        prov_agent_url = "https://academictorrents.com/details/d9e554f4f0c3047d9f49e448a7004f7aa1701b69"
        source_url = "https://doi.org/10.13003/8wx5k"
        timezone = pytz.timezone("UTC")
        current_timestamp = datetime.now(timezone).strftime('%Y-%m-%dT%H:%M:%S%z')

        # Add provenance columns
        df = df.with_columns([
            pl.lit(prov_agent_url).alias('prov_agent'),
            pl.lit(source_url).alias('source'),
            pl.lit(current_timestamp).alias('prov_date')
        ])
        return df

    def save_csv(self, df, output_csv):
        df.write_csv(output_csv)
        print(f"Updated CSV saved as {output_csv}")

class Delta:
    def __init__(self, df):
        self.df = df

    @staticmethod
    def contains_years(date_str):
        return len(date_str) >= 4

    @staticmethod
    def contains_months(date_str):
        return len(date_str) >= 7

    @staticmethod
    def contains_days(date_str):
        return len(date_str) >= 10

    @staticmethod
    def get_iso8601_duration(delta):
        duration = "P"
        if delta.years:
            duration += f"{abs(delta.years)}Y"
        if delta.months:
            duration += f"{abs(delta.months)}M"
        if delta.days:
            duration += f"{abs(delta.days)}D"
        if duration == "P":
            duration += "0D"
        return duration

    def calculate_date_difference(self, citing_date, cited_date):
        default_date = datetime(1900, 1, 1)
        
        if self.contains_years(cited_date) and self.contains_years(citing_date):
            citing_pub_datetime = parse(citing_date[:10], default=default_date)
            cited_pub_datetime = parse(cited_date[:10], default=default_date)
            delta = relativedelta(citing_pub_datetime, cited_pub_datetime)

            iso8601_duration = self.get_iso8601_duration(delta)
            if citing_pub_datetime < cited_pub_datetime: #controlla se il delta è negativo e mette il "-" davanti la P
                iso8601_duration = f"-{iso8601_duration}"
            return iso8601_duration
        else:
            return "Invalid dates"

    def add_delta_column(self):
        self.df = self.df.with_columns(
            pl.struct(["citing_date", "cited_date"]).map_elements(
                lambda x: self.calculate_date_difference(x["citing_date"], x["cited_date"]),
                return_dtype=pl.Utf8
            ).alias("time_span")
        )

    def save_csv(self, output_csv):
        self.df.write_csv(output_csv)
        print(f"CSV with Delta column saved as {output_csv}")

def main():
    parser = argparse.ArgumentParser(description="Join peer review and non-peer review DataFrames and calculate delta.")
    parser.add_argument("--peer_review_paths", help="The paths of the peer review DataFrames.", nargs='+', required=True)
    parser.add_argument("--non_peer_review_paths", help="The paths of the non-peer review DataFrames.", nargs='+', required=True)
    parser.add_argument("--output_path", help="The path of the output CSV file.", required=True)

    args = parser.parse_args()

    data_filter = Filter(args.peer_review_paths, args.non_peer_review_paths, args.output_path)
    
    concatenated_peer_df = data_filter.read_and_concatenate_dataframes(args.peer_review_paths)
    concatenated_non_peer_df = data_filter.read_and_concatenate_dataframes(args.non_peer_review_paths)
    
    data_filter.validate_dataframes(concatenated_peer_df, concatenated_non_peer_df)
    
    joined_df = data_filter.join_dataframes(concatenated_peer_df, concatenated_non_peer_df)
    
    joined_df_with_provenance = data_filter.add_provenance(joined_df)
    
    delta_calculator = Delta(joined_df_with_provenance)
    delta_calculator.add_delta_column()
    
    delta_calculator.save_csv(args.output_path)

if __name__ == "__main__":
    main()